In [1]:
from osgeo import gdal
import numpy as np

In [2]:
MODIS_min = 0
MODIS_max = 10000

S1_min = -50
S1_max = 1

S2_min = 0
S2_max = 10000

In [3]:
def tif2npy(tif_path, output_path, image_class):
    ds = gdal.Open(tif_path)
    if ds is None:
        print("No such file !")

    image = ds.ReadAsArray(0, 0, ds.RasterXSize, ds.RasterYSize)

    if image_class == "MODIS":
        image = image[[0, 1, 2, 3, 5, 6], :, :]
        # 有效范围截断
        np.clip(image, 0, 10000, out=image)
        image = (image - MODIS_min) / (MODIS_max - MODIS_min)
    elif image_class == "S1":
        image = image[[0, 1], :, :]
        # 有效范围截断
        np.clip(image, -50, 1, out=image)
        image = (image - S1_min) / (S1_max - S1_min)
    elif image_class in ["S2", "ref", "before", "after"]:
        image = image[[1, 2, 3, 4, 7, 8, 10, 11], :, :]
        # 有效范围截断
        np.clip(image, 0, 10000, out=image)
        image = (image - S2_min) / (S2_max - S2_min)
    
    # if image_class == "S2":
    #     selected_bands = [1, 2, 3, 4, 7, 8, 10, 11]
    # elif image_class == "S1":
    #     selected_bands = [0, 1]
    # elif image_class == "MODIS":
    #     selected_bands = [0, 1, 2, 3, 5, 6]

    # S2: (8, 250, 250), S1: (2, 250, 250), MODIS: (6, 5, 5)
    np.save(output_path, image.astype(np.float32))

    ds = None

    print("Successfully convert", tif_path, "to", output_path)
    
    return True

In [4]:
import os

def get_files_by_type(folder_path, file_type):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(file_type):
                file_paths.append(os.path.join(root, file))
    return file_paths

def categorize_files_by_keyword(file_paths, keywords):
    categorized_files = {keyword: [] for keyword in keywords}
    
    for file_path in file_paths:
        for keyword in keywords:
            if keyword in file_path:
                categorized_files[keyword].append(file_path)
    
    return categorized_files

folder_path = r"D:\ENVI\data\NingBo\NB_roi\cropped_data"

file_type = ".tif"

keywords = ["MODIS", "S1", "S2", "ref", "before", "after"]

file_paths = get_files_by_type(folder_path, file_type)

categorized_files = categorize_files_by_keyword(file_paths, keywords)

# for keyword, files in categorized_files.items():
#     print(f"Files with '{keyword}' in the path:")
#     for file in files:
#         print(file)
#     print("\n")


In [6]:
for keyword, files in categorized_files.items():
    for file_path in files:
        output_path = file_path.replace(r"D:\ENVI\data\NingBo\NB_roi\cropped_data", r"D:\Code\modis-s1-s2\dataset\SatelliteImages").replace("tif", "npy")
        tif2npy(file_path, output_path, image_class=keyword)

Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS\MODIS_2194.tif to D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_2194.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS\MODIS_2195.tif to D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_2195.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS\MODIS_2196.tif to D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_2196.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS\MODIS_2197.tif to D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_2197.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS\MODIS_2198.tif to D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_2198.npy
Successfully convert D:\ENVI\data\NingBo\NB_roi\cropped_data\test\MODIS\MODIS_2199.tif to D:\Code\MODIS_S1_S2\dataset\SatelliteImages\test\MODIS\MODIS_2199.npy
Successfully convert D:\ENVI\data\NingBo

In [2]:
import glob
import numpy as np


def calc_statistics(image_paths):
    images = []
    for image_path in image_paths:
        images.append(np.load(image_path))

    min_val = np.min(images, axis=(0, 2, 3))
    max_val = np.max(images, axis=(0, 2, 3))
    mean = np.mean(images, axis=(0, 2, 3))
    std = np.std(images, axis=(0, 2, 3))

    return min_val, max_val, mean, std


MODIS_dir = r"D:\Code\modis-s1-s2\dataset\SatelliteImages\train\MODIS\MODIS_*.npy"
S1_dir = r"D:\Code\modis-s1-s2\dataset\SatelliteImages\train\S1\S1_*.npy"
S2_dir = r"D:\Code\modis-s1-s2\dataset\SatelliteImages\train\S2\S2_*.npy"

MODIS_image_paths = glob.glob(MODIS_dir)
S1_image_paths = glob.glob(S1_dir)
S2_image_paths = glob.glob(S2_dir)


# min_val, max_val, mean, std = calc_statistics(MODIS_image_paths)
# print("MODIS: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

# min_val, max_val, mean, std = calc_statistics(S1_image_paths)
# print("S1: ")
# print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

min_val, max_val, mean, std = calc_statistics(S2_image_paths[:1500])
print("S2: ")
print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

S2: 
min: [0. 0. 0. 0. 0. 0. 0. 0.]
max: [1. 1. 1. 1. 1. 1. 1. 1.]
mean: [0.09811055 0.12035894 0.11604314 0.15523826 0.26191092 0.2685498
 0.2029729  0.15262435]
std: [0.05757181 0.0586444  0.06336428 0.06331751 0.11128323 0.11282161
 0.08192798 0.07514454]


In [8]:
image = np.load(r"D:\Code\MODIS_S1_S2\dataset\SatelliteImages\train\S2\S2_0.npy")

min_val = np.min(image, axis=(1, 2))
max_val = np.max(image, axis=(1, 2))
mean = np.mean(image, axis=(1, 2))
std = np.std(image, axis=(1, 2))


print(f"min: {min_val}\nmax: {max_val}\nmean: {mean}\nstd: {std}")

min: [0.067  0.0752 0.0514 0.0726 0.0667 0.0609 0.0346 0.0254]
max: [0.2618 0.3002 0.325  0.3088 0.5816 0.5784 0.3813 0.3498]
mean: [0.11357087 0.13777359 0.1343276  0.18971716 0.3307156  0.33880544
 0.2592497  0.20463623]
std: [0.02019317 0.0238121  0.04033583 0.03224437 0.07064559 0.06755491
 0.05389156 0.06252121]
